## 一、 给图像标上坐标

输入图像为 'test.jpg'

输出图像为 '1_label.jpg'

In [1]:
import cv2
import numpy as np

# 读取图像
image = cv2.imread("test.jpg")

# 转换为灰度图像
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 进行边缘检测
edges = cv2.Canny(gray, threshold1=20, threshold2=60)

# 新增
adaptive_thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY, 11, 2)

# 新增
kernel = np.ones((5, 5), np.uint8)
dilated = cv2.dilate(adaptive_thresh, kernel, iterations=1)

# 查找轮廓
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 创建一个空白画布，与原图像大小相同
output = np.copy(image)

# 存储岩石中心点坐标的列表
center_points = []

# 遍历轮廓并进行绘制和标记
for idx, contour in enumerate(contours):
    # 计算轮廓的面积
    area = cv2.contourArea(contour)
    if area > 5:  # 过滤掉较小的轮廓，防止噪声干扰
        # 绘制轮廓
        cv2.drawContours(output, [contour], -1, (0, 255, 0), 2)
        
        # 计算轮廓的中心点坐标
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            
            # 在中心点绘制一个圆
            cv2.circle(output, (cX, cY), 5, (0, 0, 255), -1)
            
            # 标注中心点坐标及编号
            cv2.putText(output, f"({cX},{cY})", (cX - 50, cY - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
            # 在中心点附近绘制坐标信息及编号
            cv2.putText(output, f"({cX},{cY})", (cX + 10, cY + 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 0), 1)
            
            # 绘制编号
            cv2.putText(output, f"{idx}", (cX - 10, cY + 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 2)
            
            # 将坐标点添加到列表中
            center_points.append((cX, cY))

cv2.imwrite("1_label.jpg", output)
# 显示绘制结果
cv2.imshow("Rock Contours and Centers", output)
cv2.waitKey(0)
cv2.destroyAllWindows()


## 二、将坐标进行打印

In [2]:
# 打印坐标点集
for point in center_points:
    print(f"坐标点: {point}")

# 将点集坐标写入文本中：要求不能有括号和逗号
with open("center_points.txt", "w") as f:
    for point in center_points:
        f.write(f"{point[0]} {point[1]}\n")
        

坐标点: (268, 1176)
坐标点: (427, 1134)
坐标点: (175, 1134)
坐标点: (395, 1123)
坐标点: (298, 1120)
坐标点: (69, 1103)
坐标点: (226, 1097)
坐标点: (473, 1086)
坐标点: (98, 1074)
坐标点: (384, 1069)
坐标点: (420, 1051)
坐标点: (286, 1052)
坐标点: (143, 1034)
坐标点: (346, 1030)
坐标点: (214, 1000)
坐标点: (88, 996)
坐标点: (425, 978)
坐标点: (334, 964)
坐标点: (120, 969)
坐标点: (490, 941)
坐标点: (281, 946)
坐标点: (123, 914)
坐标点: (390, 909)
坐标点: (181, 910)
坐标点: (245, 891)
坐标点: (79, 872)
坐标点: (433, 862)
坐标点: (352, 870)
坐标点: (165, 848)
坐标点: (513, 832)
坐标点: (262, 822)
坐标点: (499, 805)
坐标点: (413, 792)
坐标点: (175, 801)
坐标点: (317, 792)
坐标点: (91, 766)
坐标点: (253, 741)
坐标点: (452, 737)
坐标点: (294, 724)
坐标点: (214, 713)
坐标点: (511, 703)
坐标点: (377, 700)
坐标点: (172, 694)
坐标点: (210, 663)
坐标点: (256, 660)
坐标点: (342, 659)
坐标点: (102, 648)
坐标点: (251, 627)
坐标点: (200, 623)
坐标点: (376, 613)
坐标点: (173, 598)
坐标点: (108, 594)
坐标点: (438, 592)
坐标点: (321, 582)
坐标点: (526, 566)
坐标点: (56, 555)
坐标点: (231, 553)
坐标点: (340, 531)
坐标点: (398, 532)
坐标点: (317, 522)
坐标点: (103, 522)
坐标点: (221, 512)

## 三、设定攀爬合适距离

由于比例不协调的原因，先假定为 100

In [3]:
# 以2个点进行测试
from math import sqrt
A = 268 - 226
B = 1176 - 1097
print(sqrt(A**2 + B**2))

89.47066558375433


In [4]:
# 测试函数
def Compute_distance(point1, point2):
    return sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

A = [268, 1176]
B = [226, 1097]
print(Compute_distance(A, B))

89.47066558375433


In [5]:
# 计算每两个点之间的距离
for i in center_points:
    for j in center_points:
        if i != j:
            print(i, j, Compute_distance(i, j))

(268, 1176) (427, 1134) 164.45364088398895
(268, 1176) (175, 1134) 102.04410811016969
(268, 1176) (395, 1123) 137.6154061142865
(268, 1176) (298, 1120) 63.52952069707436
(268, 1176) (69, 1103) 211.96697856034086
(268, 1176) (226, 1097) 89.47066558375433
(268, 1176) (473, 1086) 223.8861317723811
(268, 1176) (98, 1074) 198.25236442474022
(268, 1176) (384, 1069) 157.8131806916013
(268, 1176) (420, 1051) 196.7968495682794
(268, 1176) (286, 1052) 125.29964086141668
(268, 1176) (143, 1034) 189.17980864775183
(268, 1176) (346, 1030) 165.5294535724685
(268, 1176) (214, 1000) 184.09780009549272
(268, 1176) (88, 996) 254.55844122715712
(268, 1176) (425, 978) 252.69151153135317
(268, 1176) (334, 964) 222.03603311174518
(268, 1176) (120, 969) 254.46610776290032
(268, 1176) (490, 941) 323.2785176902418
(268, 1176) (281, 946) 230.3670983452281
(268, 1176) (123, 914) 299.44782517159814
(268, 1176) (390, 909) 293.5523803344132
(268, 1176) (181, 910) 279.8660393831306
(268, 1176) (245, 891) 285.9265639

## 三、当距离小于定值进行连边

In [6]:
import cv2
import numpy as np

# 计算欧几里得距离
def euclidean_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

# 读取图像
image = cv2.imread("test.jpg")

# 转换为灰度图像
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 进行边缘检测
edges = cv2.Canny(gray, threshold1=20, threshold2=60)

# 新增
adaptive_thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY, 11, 2)

# 新增
kernel = np.ones((5, 5), np.uint8)
dilated = cv2.dilate(adaptive_thresh, kernel, iterations=1)

# 查找轮廓
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 创建一个空白画布，与原图像大小相同
output = np.copy(image)

# 存储岩石中心点坐标的列表
center_points = []

# 遍历轮廓并进行绘制和标记
for idx, contour in enumerate(contours):
    # 计算轮廓的面积
    area = cv2.contourArea(contour)
    if area > 5:  # 过滤掉较小的轮廓，防止噪声干扰
        # 绘制轮廓
        cv2.drawContours(output, [contour], -1, (0, 255, 0), 2)
        
        # 计算轮廓的中心点坐标
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            
            # 在中心点绘制一个圆
            cv2.circle(output, (cX, cY), 5, (0, 0, 255), -1)
            
            # 标注中心点坐标及编号
            cv2.putText(output, f"({cX},{cY})", (cX - 50, cY - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
            # 在中心点附近绘制坐标信息及编号
            cv2.putText(output, f"({cX},{cY})", (cX + 10, cY + 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 0), 1)
            
            # 绘制编号
            cv2.putText(output, f"{idx}", (cX - 10, cY + 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            
            # 将坐标点添加到列表中
            center_points.append((cX, cY))

# 绘制蓝色连边
for i in range(len(center_points)):
    for j in range(i + 1, len(center_points)):
        distance = euclidean_distance(center_points[i], center_points[j])
        if distance < 90:
            # cv2.line(output, center_points[i], center_points[j], (255, 0, 0), 1)  # 修改颜色为蓝色

cv2.imwrite("1_label_with_blue_edges_90.jpg", output)
# 显示绘制结果
cv2.imshow("Rock Contours, Centers, and Blue Edges", output)
cv2.waitKey(0)
cv2.destroyAllWindows()


IndentationError: expected an indented block after 'if' statement on line 70 (1758570528.py, line 73)

## 四、Dijkstra最短路算法

In [7]:
import numpy as np
import heapq

# 计算欧几里得距离
def euclidean_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

# Dijkstra算法计算最短路径
def dijkstra(graph, start):
    distances = {vertex: float('infinity') for vertex in graph}
    distances[start] = 0
    priority_queue = [(0, start)]

    while priority_queue:
        current_distance, current_vertex = heapq.heappop(priority_queue)

        if current_distance > distances[current_vertex]:
            continue

        for neighbor, weight in graph[current_vertex].items():
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))

    return distances

In [37]:
import cv2
import numpy as np
import heapq

def euclidean_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def dijkstra(graph, start):
    distances = {vertex: float('infinity') for vertex in graph}
    distances[start] = 0
    priority_queue = [(0, start)]

    while priority_queue:
        current_distance, current_vertex = heapq.heappop(priority_queue)

        if current_distance > distances[current_vertex]:
            continue

        for neighbor, weight in graph[current_vertex].items():
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))

    return distances

title = "using 70"
cv2.putText(output, title, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 0, 255), 5)

image = cv2.imread("test.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, threshold1=20, threshold2=60)
adaptive_thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY, 11, 2)
kernel = np.ones((5, 5), np.uint8)
dilated = cv2.dilate(adaptive_thresh, kernel, iterations=1)
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

output = np.copy(image)
center_points = []

index = 1
for idx, contour in enumerate(contours):
    area = cv2.contourArea(contour)
    if area > 5:
        M = cv2.moments(contour)
        # if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        cv2.circle(output, (cX, cY), 5, (0, 0, 255), -1)
        cv2.putText(output, f"({cX},{cY})", (cX - 50, cY - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.putText(output, f"({cX},{cY})", (cX + 10, cY + 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 0), 1)
        cv2.putText(output, f"{index}", (cX - 10, cY + 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        center_points.append((cX, cY))
        index = index + 1

graph = {}
for i in range(len(center_points)):
    graph[i] = {}
    for j in range(i + 1, len(center_points)):
        distance = euclidean_distance(center_points[i], center_points[j])
        if distance <= 100:
            graph[i][j] = distance
            # cv2.line(output, center_points[i], center_points[j], (255, 0, 0), 1)  # 修改颜色为蓝色

shortest_paths = dijkstra(graph, start=1)

target_points = [181, 174, 182]
for target in target_points:
    if target in shortest_paths:
        print(f"Shortest distance from point 1 to point {target}: {shortest_paths[target]:.2f}")
    else:
        print(f"No path from point 1 to point {target}")

cv2.imwrite("label.jpg", output)
cv2.imshow("Rock Contours, Centers, and Blue Edges", output)
cv2.waitKey(0)
cv2.destroyAllWindows()


No path from point 1 to point 181
No path from point 1 to point 174
No path from point 1 to point 182


In [10]:
# 将center_points写入文本文件中
file_name = "center_points_70.txt"

# 打开文件以写入模式
with open(file_name, "w") as file:
    for point in center_points:
        # 将每个元组的数据转换为字符串，然后写入文件
        file.write(f"{point[0]} {point[1]}\n")

print("列表已成功写入文本文件。")

列表已成功写入文本文件。


In [18]:
import cv2
import numpy as np
import heapq

image = cv2.imread("test.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, threshold1=20, threshold2=60)
adaptive_thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY, 11, 2)
kernel = np.ones((5, 5), np.uint8)
dilated = cv2.dilate(adaptive_thresh, kernel, iterations=1)
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

output = np.copy(image)
center_points = []

target_points = [1,7,15,24,29,34,40,49,57,62,69,78,88,91,100,108,111]

for idx, contour in enumerate(contours):
    area = cv2.contourArea(contour)
    if area > 5:
        M = cv2.moments(contour)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])

        if idx in target_points:
            cv2.circle(output, (cX, cY), 5, (0, 0, 0), -1)
        
        cv2.putText(output, f"({cX},{cY})", (cX - 50, cY - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.putText(output, f"({cX},{cY})", (cX + 10, cY + 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 0), 1)
        cv2.putText(output, f"{idx}", (cX - 10, cY + 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        center_points.append((cX, cY))

for i, point_idx in enumerate(target_points):
    for neighbor_idx in target_points[i + 1:]:
        cv2.line(output, center_points[point_idx], center_points[neighbor_idx], (0, 0, 0), 2)

cv2.imshow("Rock Contours, Centers, and Black Lines", output)
cv2.waitKey(0)
cv2.destroyAllWindows()
